In [1]:
import os
import sys
import math
import uproot
import numpy as np

import matplotlib.pyplot as plt

from scipy          import stats
from scipy.optimize import curve_fit
from collections    import namedtuple

In [2]:
### Import functions from peak_functions

repository_path = os.path.abspath('/Users/romoluque_c/Repositories/BACON_romo/')
sys.path.append(repository_path)

import peak_functions as pf

In [3]:
plt.rcParams["figure.figsize"] = 9, 6
plt.rcParams["font.size"]      = 13

## Calibration runs taken in September

In [4]:
data_path = '/Users/romoluque_c/LEGEND/BACON/datatest/SiPM_tests_sept/'

In [5]:
def load_data(channel, day, vbias, vled):
    return uproot.open(data_path + f'QE{channel}-09_{day}_2023-V_{vbias}-A_{vled}-Hz_10.root')['RawTree']

def compute_max_wf(RawTree, channel=0):
    wfs_chann   = np.array(RawTree[f'chan{channel}/rdigi'].array())
    subt_raw_wf = list(map(pf.subtract_baseline, wfs_chann))
    max_wf      = np.array([np.max(subt_raw_wf[evt][int(1550/2):int(1700/2)]) for evt in range(len(subt_raw_wf))])
    return max_wf

def load_data_and_compute_max_wf(channel, day, vbias, vled):
    data   = load_data(channel, day, vbias, vled)
    max_wf = compute_max_wf(data)
    return max_wf

In [6]:
RawTree1  = load_data(channel=1,  day='01', vbias=55, vled=7.4)
RawTree2  = load_data(channel=2,  day='01', vbias=55, vled=7.4)
RawTree65 = load_data(channel=65, day='07', vbias=55, vled=6.4)

In [ ]:
channels = [   1,    2,    3,   55,   56,   57,   58,   59,   60,   64,   65,   66]
vleds0   = [   7,    7,    7,    6,    6,    6,    6,    6,    6,    6,    6,    6]
days     = ['01', '01', '01', '11', '11', '11', '11', '11', '11', '07', '07', '07']

channel_v    = namedtuple('channel', 'channel vbias vled')
all_max_data = {channel_v(channel=ch, vbias=vb, vled=vl): load_data_and_compute_max_wf(ch, day, vb, vl)
                for ch, vled0, day in zip(channels, vleds0, days)
                for vl             in [vled0 + 0.4, vled0 + 0.6]
                for vb             in [55, 56]}

### Take a look at multiple waveforms of the same SiPM

In [ ]:
channel = 1
plt.figure(figsize=(6, 4.5))
for evt in range(10):
    wf2 = np.array(RawTree1[f'chan0/rdigi'].array())[evt]
    plt.plot(2*np.arange(len(wf2)), wf2, linewidth=0.5)
plt.xlabel('Time window (ns)', fontsize=14)
plt.ylabel('Amplitude (ADCs)', fontsize=14)
plt.title(f"Channel {channel}")
plt.tight_layout()
plt.show()

In [ ]:
channel = 1
plt.figure(figsize=(7, 4.5))
for evt in range(1):
    wf2 = np.array(RawTree1[f'chan0/rdigi'].array())[evt]
    plt.plot(2*np.arange(len(wf2)), wf2, linewidth=0.5)
plt.xlabel('Time window (ns)', fontsize=14)
plt.ylabel('Amplitude (ADCs)', fontsize=14)
#plt.xlim(0,2000)
plt.title(f"Channel {channel}")
plt.tight_layout()
plt.show()

In [ ]:
channel  = 1
num_evts = 100
plt.figure(figsize=(7, 4.5))
sum_wf2 = np.sum(np.array(RawTree1[f'chan0/rdigi'].array())[:], axis=0)
plt.plot(2*np.arange(len(wf2)), sum_wf2, linewidth=0.5)
plt.xlabel('Time window (ns)', fontsize=14)
plt.ylabel('Summed wf Amplitude (ADCs)', fontsize=14)
#plt.xlim(0,2000)
plt.title(f"Channel {channel}")
plt.tight_layout()
plt.show()

In [ ]:
channel  = 1
num_evts = 100
plt.figure(figsize=(7, 4.5))
sum_wf2 = np.sum(np.array(RawTree1[f'chan0/rdigi'].array()), axis=0)
plt.plot(2*np.arange(len(wf2)), sum_wf2, linewidth=0.5)
plt.axvspan(1550, 1700, color='grey', alpha=0.2)
plt.xlabel('Time window (ns)', fontsize=14)
plt.ylabel('Summed wf Amplitude (ADCs)', fontsize=14)
plt.xlim(1200,2200)
plt.title(f"Channel {channel}")
plt.tight_layout()
plt.show()

In [ ]:
channel = 1
for evt in range(10):
    plt.figure(figsize=(6, 4.5))
    wf2 = np.array(RawTree1[f'chan0/rdigi'].array())[evt]
    plt.plot(2*np.arange(len(wf2)), wf2, linewidth=0.5)
    plt.xlabel('Time window (ns)', fontsize=14)
    plt.ylabel('Amplitude (ADCs)', fontsize=14)
    plt.xlim(1200,2000)
    plt.title(f"Channel {channel}")
    plt.tight_layout()
    plt.show()

### Analyze spectra

In [ ]:
plt.hist(all_max_data[channel_v(channel=1, vbias=55, vled=7.6)], bins=150, range=(0, 1500), alpha=0.6, label='V$_{\mathrm{bias}}$ = 55 V, V$_{\mathrm{LED}}$ = 7.6 V')
plt.hist(all_max_data[channel_v(channel=1, vbias=56, vled=7.6)], bins=150, range=(0, 1500), alpha=0.6, label='V$_{\mathrm{bias}}$ = 56 V, V$_{\mathrm{LED}}$ = 7.6 V')
plt.title('Channel1')
plt.xlabel('Amplitude (ADCs)', fontsize=14)
plt.ylabel('Entries/bin',      fontsize=14)
#plt.xlim(0, 1500)
plt.legend()
plt.show()

In [ ]:
plt.hist(all_max_data[channel_v(channel=1, vbias=56, vled=7.4)], bins=150, range=(0, 1500), alpha=0.6, label='V$_{\mathrm{bias}}$ = 56 V, V$_{\mathrm{LED}}$ = 7.4 V')
plt.hist(all_max_data[channel_v(channel=1, vbias=56, vled=7.6)], bins=150, range=(0, 1500), alpha=0.6, label='V$_{\mathrm{bias}}$ = 56 V, V$_{\mathrm{LED}}$ = 7.6 V')
plt.title('Channel 1')
plt.xlabel('Amplitude (ADCs)', fontsize=14)
plt.ylabel('Entries/bin',      fontsize=14)
#plt.xlim(0, 1500)
plt.legend()
plt.show()

In [ ]:
plt.hist(all_max_data[channel_v(channel=1, vbias=55, vled=7.4)], bins=150, range=(0, 1500), alpha=0.6, label='Channel1')
plt.hist(all_max_data[channel_v(channel=2, vbias=55, vled=7.4)], bins=150, range=(0, 1500), alpha=0.6, label='Channel2')
plt.hist(all_max_data[channel_v(channel=3, vbias=55, vled=7.4)], bins=150, range=(0, 1500), alpha=0.6, label='Channel3')
plt.title('V$_{\mathrm{bias}}$ = 55 V, V$_{\mathrm{LED}}$ = 7.4 V')
plt.xlabel('Amplitude (ADCs)', fontsize=14)
plt.ylabel('Entries/bin',      fontsize=14)
#plt.xlim(0, 1500)
plt.legend()
plt.show()

In [ ]:
plt.hist(all_max_data[channel_v(channel=1, vbias=55, vled=7.6)], bins=150, range=(0, 1300), alpha=0.6, label='Channel1')
plt.hist(all_max_data[channel_v(channel=2, vbias=55, vled=7.6)], bins=150, range=(0, 1300), alpha=0.6, label='Channel2')
plt.hist(all_max_data[channel_v(channel=3, vbias=55, vled=7.6)], bins=150, range=(0, 1300), alpha=0.6, label='Channel3')
plt.title('V$_{\mathrm{bias}}$ = 55 V, V$_{\mathrm{LED}}$ = 7.6 V')
plt.xlabel('Amplitude (ADCs)', fontsize=14)
plt.ylabel('Entries/bin',      fontsize=14)
#plt.xlim(0, 1500)
plt.legend()
plt.show()

In [ ]:
vled = 6.6
vb   = 55
for ch in [55, 56, 57, 58, 59, 60, 64, 65, 66]:
    plt.hist(all_max_data[channel_v(channel=ch, vbias=vb, vled=vled)], bins=120, range=(0, 1500), alpha=0.6, label=f'Channel {ch}')
plt.title('V$_{\mathrm{bias}}$ = 55 V, V$_{\mathrm{LED}}$ = 6.6 V')
plt.xlabel('Amplitude (ADCs)', fontsize=14)
plt.ylabel('Entries/bin',      fontsize=14)
#plt.xlim(0, 1500)
plt.legend()
plt.show()

In [ ]:
vled = 6.6
vb   = 56
for ch in [55, 56, 57, 58, 59, 60, 64, 65, 66]:
    plt.hist(all_max_data[channel_v(channel=ch, vbias=vb, vled=vled)], bins=150, range=(0, 1500), alpha=0.6, label=f'ich {ch}')
plt.title(f'V_{vb}, A_{vled}')
plt.xlabel('Amplitude (ADCs)', fontsize=14)
plt.ylabel('Entries/bin',      fontsize=14)
#plt.xlim(0, 1500)
plt.legend()
plt.show()

### Fit

In [ ]:
# Define the function for a single Gaussian peak
def gaussian(x, amplitude, mean, stddev):
    return amplitude * np.exp(-(x - mean)**2 / (2 * stddev**2))

# Define the function for multiple Gaussian peaks
def multi_gaussian(x, *params):
    n_peaks = len(params) // 3
    y       = np.zeros_like(x)
    for i in range(n_peaks):
        amplitude = params[i * 3]
        mean      = params[i * 3 + 1]
        stddev    = params[i * 3 + 2]
        y += gaussian(x, amplitude, mean, stddev)
    return y

def shift_to_bin_centers(x):
    """
    Return bin centers, given bin lower edges.
    """
    return x[:-1] + np.diff(x) * 0.5

def truncate(number, decimals=0):
    """
    Returns a value truncated to a specific number of decimal places.
    """
    if not isinstance(decimals, int):
        raise TypeError("decimal places must be an integer.")
    elif decimals < 0:
        raise ValueError("decimal places has to be 0 or more.")
    elif decimals == 0:
        return math.trunc(number)

    factor = 10.0 ** decimals
    return math.trunc(number * factor) / factor

def plot_linear_fit(y, yerr):
    x    = np.arange(len(y))+1
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    line = slope*x + intercept
    
    fig = plt.figure(figsize=(8,5))
    plt.errorbar(x, y, yerr=yerr, marker='_', markersize=5, linestyle='', c='k', label=f'Measured gain values')
    plt.plot(x, line, color='r', alpha=0.7, label=f'Fit: y = x*{round(slope, 2)} - {round(np.abs(intercept), 2)}, \n     R$^2$ = {truncate(r_value, 2)}')
    
    plt.xlabel('Peak number')
    plt.ylabel('Mu from fit (ADCs)')
    plt.legend(fontsize=14, loc='upper left')
    plt.show()
    return slope

In [ ]:
all_max_data

### Fit spectra

In [ ]:
def extract_gain_and_mean(channel, vbias, vled, bins, prange, initial_guess):
    
    data = all_max_data[channel_v(channel=channel, vbias=vbias, vled=vled)]
    
    print()
    print('------------------------------------------------------------------------------------------')
    print(f'-------    Channel {channel}, Vbias = {vbias} V, VLED = {vled}   ------------------------')
    print('------------------------------------------------------------------------------------------')
    print()

    plt.figure(figsize=(7, 5))
    y, x, _ = plt.hist(data, bins=bins, range=prange, alpha=0.6, label=f'Channel{channel}')
    #plt.axvline(x=740, color='r')
    plt.xlabel('Amplitude (ADCs)', fontsize=15)
    plt.ylabel('Entries/bin',      fontsize=15)
    plt.title(f"Spectrum for channel {channel} (height of the peaks)", fontsize=15)
    plt.show()
    
    popt, pcov = curve_fit(multi_gaussian, shift_to_bin_centers(x), y, p0=initial_guess)
    
    plt.figure(figsize=(7, 5))
    plt.step(shift_to_bin_centers(x), y, label='Spectrum')
    plt.plot(x, multi_gaussian(x, *popt), 'r--', label='Fit')
    plt.xlabel('Amplitude (ADCs)', fontsize=15)
    plt.ylabel('Entries/bin',      fontsize=15)
    plt.title(f"Channel {channel}",     fontsize=15)
    plt.legend()
    plt.show()
    
    perr         = np.sqrt(np.diag(pcov))    
    means_ch     = np.array([popt[i*3+1] for i in range(len(popt)//3)])
    means_ch_err = np.array([perr[i*3+1] for i in range(len(perr)//3)])
    
    gain_ch = plot_linear_fit(means_ch[:4], means_ch_err[:4])
    
    return gain_ch, means_ch

In [ ]:
all_gains = {}
all_means = {}

In [ ]:
vb = 56
for ch, vled in zip(channels, vleds0):
    vl = vled + 0.6
    initial_guess  = [200, 20, 10, 250, 200, 15, 120, 400, 15, 60, 550, 15, 15, 740, 10]
    gain, ch_means = extract_gain_and_mean(channel       = ch,
                                           vbias         = vb,
                                           vled          = vl,
                                           bins          = 120,
                                           prange        = (-50, 1500),
                                           initial_guess = initial_guess)
    
    all_gains[channel_v(channel=ch, vbias=vb, vled=vl)] = gain
    all_means[channel_v(channel=ch, vbias=vb, vled=vl)] = ch_means

In [ ]:
vb = 55
for ch, vled in zip(channels, vleds0):
    vl = vled + 0.6
    initial_guess = [200, 20, 10, 250, 155, 15, 120, 290, 15, 60, 430, 15, 15, 580, 10]
    gain, ch_means = extract_gain_and_mean(channel       = ch,
                                           vbias         = vb,
                                           vled          = vl,
                                           bins          = 100,
                                           prange        = (-50, 1200),
                                           initial_guess = initial_guess)
    all_gains[channel_v(channel=ch, vbias=vb, vled=vl)] = gain
    all_means[channel_v(channel=ch, vbias=vb, vled=vl)] = ch_means

In [ ]:
gains_vb55 = np.array([all_gains[channel_v(channel=ch, vbias=55, vled=0.6+vl)] for ch,vl in zip(channels, vleds0)])
gains_vb56 = np.array([all_gains[channel_v(channel=ch, vbias=56, vled=0.6+vl)] for ch,vl in zip(channels, vleds0)])

In [ ]:
plt.scatter(range(len(gains_vb55)), gains_vb55, marker='*', s=50, label='V$_{\mathrm{bias}}$ = 55 V')
plt.scatter(range(len(gains_vb56)), gains_vb56, marker='*', s=50, label='V$_{\mathrm{bias}}$ = 56 V')
plt.xlim(-1, 12)
plt.ylim(70, 250)
plt.ylabel('Gain (ADCs)')
plt.xlabel('Channel')
plt.xticks(range(len(gains_vb55)), channels)
plt.legend()
plt.show()

In [ ]:
lab1 = 'V$_{\mathrm{bias}}$ = 55 V \n$\mu$ = '+f'{np.round(np.mean(gains_vb55), 1)} ADCs \n$\sigma$ = {np.round(np.std(gains_vb55), 1)} ADCs'
lab2 = 'V$_{\mathrm{bias}}$ = 56 V \n$\mu$ = '+f'{np.round(np.mean(gains_vb56), 1)} ADCs \n$\sigma$ = {np.round(np.std(gains_vb56), 1)} ADCs'

plt.hist(gains_vb55, bins=70, range=(90, 230), label=lab1, alpha=0.7)
plt.hist(gains_vb56, bins=70, range=(90, 230), label=lab2, alpha=0.7)
plt.xlabel('Gain (ADCs)')
plt.ylabel('Entries/bin')
plt.legend()
plt.show()

### Compute the quantum efficiency for the different channels

In [ ]:
colors = ['indianred', 'grey', 'orange', 'skyblue']

plt.figure(figsize=(7, 5))
y, x, _ = plt.hist(all_max_data[channel_v(channel=1, vbias=55, vled=7.6)], bins=150, range=(-50, 1000), alpha=0.6, color='teal')
means_ch1 = all_means[channel_v(channel=1, vbias=55, vled=7.6)]
gain_ch1  = all_gains[channel_v(channel=1, vbias=55, vled=7.6)]

for i, (mean, col) in enumerate(zip(means_ch1[:3], colors[:3])):
    plt.axvspan(mean-gain_ch1/2, mean+gain_ch1/2, alpha=0.2, color=col, label=f'Peak {i}')
plt.axvspan(means_ch1[2]+gain_ch1/2, means_ch1[2]+3*gain_ch1/2, alpha=0.2, color='skyblue', label=f'Peak 3')
plt.xlabel('Amplitude (ADCs)', fontsize=15)
plt.ylabel('Entries/bin',      fontsize=15)
plt.title(f"Channel 1", fontsize=15)
plt.legend()
plt.show()

In [ ]:
integ_all_chs = {}

for vb in [55, 56]:
    for ch,vl in zip(channels, vleds0):
        means_ch1 = all_means[channel_v(channel=ch, vbias=vb, vled=0.6+vl)]
        gain_ch1  = all_gains[channel_v(channel=ch, vbias=vb, vled=0.6+vl)]
        ints_ch   = []
        for mean_ch in means_ch1[:3]:
            ch_spec    = all_max_data[channel_v(channel=ch, vbias=vb, vled=0.6+vl)]
            peak_wf_rt = np.sum(ch_spec[(ch_spec>(mean_ch-gain_ch1/2)) & (ch_spec<(mean_ch+gain_ch1/2))])
            ints_ch.append(peak_wf_rt)
            
        ints_ch.append(np.sum(ch_spec[(ch_spec>(means_ch1[2]+gain_ch1/2)) & (ch_spec<(means_ch1[2]+3*gain_ch1/2))]))
        
        integ_all_chs[channel_v(channel=ch, vbias=vb, vled=0.6+vl)] = np.array(ints_ch)

In [ ]:
for vb in [55, 56]:
    plt.figure(figsize=(7, 5))
    for ch,vl in zip(channels, vleds0):
        ints_ch = integ_all_chs[channel_v(channel=ch, vbias=vb, vled=0.6+vl)]
        #print(np.arange(4)*ints_ch/ints_ch1)
        if ch < 4:
            ref_ch = integ_all_chs[channel_v(channel=1,  vbias=vb, vled=7.6)]
        else:
            ref_ch = integ_all_chs[channel_v(channel=55, vbias=vb, vled=6.6)]
        plt.scatter(np.arange(4), np.arange(4)*ints_ch/ref_ch, marker='*', label=f'Channel {ch}')
    plt.legend(fontsize=10, ncol=2)
    plt.show()

In [ ]:
for vb in [55, 56]:
    plt.figure(figsize=(7, 5))
    for ch,vl in zip(channels, vleds0):
        ints_ch = integ_all_chs[channel_v(channel=ch, vbias=vb, vled=0.6+vl)]
        #print(np.arange(4)*ints_ch/ints_ch1)
        if ch < 4:
            ref_ch = integ_all_chs[channel_v(channel=1,  vbias=vb, vled=7.6)]
            plt.scatter(np.arange(4), np.arange(4)*ints_ch/ref_ch, marker='*', s=60, label=f'Channel {ch}')
    plt.legend(fontsize=12)
    plt.title(f'Vbias = {vb} V')
    plt.xlim(-0.5, 3.5)
    plt.ylim(-1, 4)
    plt.xlabel('Peak number')
    plt.ylabel('Relative QE between channels')
    plt.show()

In [ ]:
for vb in [55, 56]:
    plt.figure(figsize=(7, 5))
    for ch,vl in zip(channels, vleds0):
        ints_ch = integ_all_chs[channel_v(channel=ch, vbias=vb, vled=0.6+vl)]
        #print(np.arange(4)*ints_ch/ints_ch1)
        if ch > 4:
            ref_ch = integ_all_chs[channel_v(channel=55,  vbias=vb, vled=6.6)]
            plt.scatter(np.arange(4), np.arange(4)*ints_ch/ref_ch, marker='*', s=60, label=f'Channel {ch}')
    plt.title(f'Vbias = {vb} V')
    plt.legend(fontsize=12, ncol=2)
    plt.xlim(-0.5, 3.5)
    plt.ylim(-1, 5)
    plt.xlabel('Peak number')
    plt.ylabel('Relative QE between channels')
    plt.show()